In [46]:
import re

import requests
import pdfplumber
import pandas as pd
from collections import namedtuple
import fitz

In [47]:
Inv = namedtuple('Inv', 'vend_num vend_name inv_dt due_dt inv_amt net_amt description')

In [48]:
def download_file(url):
    local_filename = url.split('/')[-1]
    
    with requests.get(url) as r:
        with open(local_filename, 'wb') as f:
            f.write(r.content)
        
    return local_filename

In [49]:
ap_url = 'https://www.tabs3.com/support/sample/apreports.pdf'

In [50]:
ap = download_file(ap_url)
print(type(ap))
ap =fitz.open("189641.pdf")  
ap = str("189641.pdf")
print(type(ap))

<class 'str'>
<class 'str'>


In [51]:
with pdfplumber.open(ap) as pdf:
    page = pdf.pages[11]
    text = page.extract_text()
    

In [52]:
text

' \n   \n \nD=2.5T+R  D=4T+R\n \n   \nDrawing: \nShapes-A myriad of shapes can be formed through multiple draws.  Round shapes offer that \ngreatest ease and economy in drawing.  Then next best option is square with adequate corner \nradii.  Costs increase for combinations of basic shapes and irregular shapes. \n \nRadii-To facilitate drawing, keep radii as generous as possible:  punch and die radius should be a \nminimum of four times the material thickness.  The part radius should be at no less than six times \nthe material thickness with appropriate drawing-quality material. \n \n \n \n  - 12 - '

In [53]:
new_vend_re = re.compile(r'^\d{3} [A-Z].*')

In [54]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        print(line)

In [55]:
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)


In [56]:
inv_line_re = re.compile(r'(\d{6}) (\d{6}) ([\d,]+\.\d{2}) [\sP]*([\d,]+\.\d{2}) [YN ]*\d (.*?) [*\s\d]')

In [57]:
line_items = []
for line in text.split('\n'):
    if new_vend_re.match(line):
        vend_num, *vend_name = line.split()
        vend_name = ' '.join(vend_name)    
    
    line = inv_line_re.search(line)
    if line:
        inv_dt = line.group(1)
        due_dt = line.group(2)
        inv_amt = line.group(3)
        net_amt = line.group(4)
        desc = line.group(5)
        line_items.append(Inv(vend_num, vend_name, inv_dt, due_dt, inv_amt, net_amt, desc))

In [58]:
df = pd.DataFrame(line_items)

In [61]:
df.head()

""


In [60]:
df['inv_amt'] = df['inv_amt'].map(lambda x: float(x.replace(',', '')))
df['net_amt'] = df['net_amt'].map(lambda x: float(x.replace(',', '')))
df.sum()

KeyError: 'inv_amt'

In [ ]:
df.to_csv('inv_lines.csv')